In [1]:
import pandas as pd
import numpy as np
import numbers
import decimal
import scipy.stats as ss
import matplotlib.pyplot as plt
from statistics import stdev
from statistics import mean
import time
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

In [2]:
redditDataTrain = pd.read_csv("data/reddit_train.csv") #, sep="\n", header=None) 
redditDataTest = pd.read_csv("data/reddit_test.csv") # sep="\n", header=None)

In [854]:
redditDataTrain

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny
...,...,...,...
69995,69995,"Thank you, you confirm Spain does have nice pe...",europe
69996,69996,Imagine how many he would have killed with a r...,leagueoflegends
69997,69997,Yes. Only. As in the guy I was replying to was...,canada
69998,69998,Looking for something light-hearted or has a v...,anime


In [5]:
allClasses = ["hockey", "nba", "leagueoflegends", "soccer", "funny", "movies", "anime", "Overwatch", "trees", "GlobalOffensive", "nfl", "AskReddit", "gameofthrones", "conspiracy", "worldnews", "wow", "europe", "canada", "Music", "baseball"]

In [662]:
x_train, x_test, y_train, y_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2, random_state=4)

In [855]:
x_train

31604    Irrational silly people who call themselves St...
10793    Last time I checked none of them have champion...
52093    feels rlly bad brushing after drinking coke yo...
25134    If anything it has very little to do with the ...
31569    It's about 30-40 km east of Winnipeg. There is...
                               ...                        
46393    1 minute build up with a single four second ca...
20062    Because that's the corner they've painted them...
65276          I wish but I can't watch what he's from ;n;
23346    He saved you the trouble of typing out somethi...
11863    First time viewer here. I feel a bit sorry for...
Name: comments, Length: 56000, dtype: object

In [7]:
commentsTrain = redditDataTrain.iloc[:,1]
subredditsTrain = redditDataTrain.iloc[:,-1]
commentsTest = redditDataTest.iloc[:,1]

In [16]:
subredditsTrain.unique()

array(['hockey', 'nba', 'leagueoflegends', 'soccer', 'funny', 'movies',
       'anime', 'Overwatch', 'trees', 'GlobalOffensive', 'nfl',
       'AskReddit', 'gameofthrones', 'conspiracy', 'worldnews', 'wow',
       'europe', 'canada', 'Music', 'baseball'], dtype=object)

In [20]:
tfidf = TfidfVectorizer(stop_words='english')
cv = CountVectorizer()
lr = LogisticRegression()
multiNB = MultinomialNB()
dtc = tree.DecisionTreeClassifier()
kf = KFold(n_splits=5)


In [664]:
redditDataTrainTF = tfidf.fit_transform(x_train)
redditDataTestTF = tfidf.transform(x_test)

In [143]:
def getScoretWithModel(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)


In [668]:
multiNB.fit(redditDataTrainTF[:4], y_train[:4])
multiNB.predict(redditDataTestTF[:4])


array(['GlobalOffensive', 'GlobalOffensive', 'GlobalOffensive', 'hockey'],
      dtype='<U15')

In [144]:
print(getScoretWithModel(lr, redditDataTrainTF, redditDataTestTF, y_train, y_test))

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5506428571428571


In [146]:
print(getScoretWithModel(multiNB, redditDataTrainTF, redditDataTestTF, y_train, y_test))

0.5637142857142857


In [906]:
cv = CountVectorizer(binary=True, stop_words='english')
X= x_train[:3]
# X.append(pd.Series(x_train[2]))
X = cv.fit_transform(x_train[:3])
y = y_train[:3]
# y.append(y_train[2])
y.values

array(['nfl', 'hockey', 'GlobalOffensive'], dtype=object)

In [956]:
x=X[0]

In [932]:
indices = np.where(y==k)[0]
filteredX = X[[0,2]]
filteredX.A1

AttributeError: A1 not found

In [969]:
k='nfl'
thetajk[k] = (filteredX.sum(axis=0))/filteredX.shape[0]
# thetajk[k]
# float(filteredX.shape[0])+2
y = thetajk[k].dot(x.T)
y

matrix([[<25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
         <25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
         <25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
         <25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
         <25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
         <25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
         <25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
         <25x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Column format>,
        

In [606]:
from collections import defaultdict
from functools import partial
thetak = defaultdict(float)
thetajk = defaultdict(partial(np.ndarray, 0))

In [880]:
filteredX.toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0]])

In [656]:
for k in y.values:
    thetak[k] = (y==k).sum()/float(y.shape[0])
    indices = np.where(y==k)[0]
    filteredX = X[indices]
    thetajk[k] = (filteredX.toarray().sum(axis=0)+1)/(float(filteredX.shape[0]+2))
print(thetak, thetajk['nfl'])

defaultdict(<class 'float'>, {'nfl': 0.3333333333333333, 'hockey': 0.3333333333333333, 'GlobalOffensive': 0.3333333333333333}) [0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
 0.33333333 0.33333333 0.66666667 0.33333333 0.66666667 0.33333333
 0.33333333 0.33333333 0.66666667 0.33333333 0.66666667 0.33333333
 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667 0.33333333
 0.33333333]


In [660]:
classes=["nfl", "hockey", "GlobalOffensive"]
class_prob=[]
for k in classes:
    likelihood=0
    for j in range(len(Xtest)):
        likelihood += Xtest[j]*np.log(thetajk[k][j]) + (1-Xtest[j])*(np.log(1-thetajk[k][j]))
    class_prob.append(likelihood + np.log(thetak[k]))
class_prob
# Xtest[j]*np.log(thetajk[k][j]) + (1-Xtest[j])*(np.log(1-thetajk[k][j]))

[-16.08727025529183, -18.859858977531612, -17.47356461641172]

In [675]:
np.log(0.33333333)

-1.0986122986681097

In [705]:
Xtest = redditDataTestTF[:4].toarray()[0]
Xtest

array([0., 0., 0., ..., 0., 0., 0.])

In [707]:
thetajk[k]

array([0.33333333, 0.66666667, 0.33333333, 0.66666667, 0.33333333,
       0.33333333, 0.66666667, 0.66666667, 0.33333333, 0.66666667,
       0.33333333, 0.33333333, 0.66666667, 0.33333333, 0.33333333,
       0.66666667, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
       0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333])

In [708]:
len(Xtest)

65889

In [706]:
test=0
for j in range(len(Xtest)):
    test += (1-Xtest[j])*(np.log(1-thetajk[k][j]))

test

IndexError: index 25 is out of bounds for axis 0 with size 25

In [856]:
from collections import defaultdict
from functools import partial

class MultiNomialNB:
    def __init__(self):
        self.thetak = defaultdict(float)
        self.thetajk = defaultdict(partial(np.ndarray, 0))
        self.classes = y.values
        
    def fit(self, X, y):
        for k in y.values:
            self.thetak[k] = (y==k).sum()/float(y.shape[0])
            indices = np.where(y==k)[0]
            filteredX = X[indices]
            self.thetajk[k] = (filteredX.toarray().sum(axis=0)+1)/(float(filteredX.shape[0]+2))
        print(self.thetak, len(self.thetajk[k]))
    
    def predict(self, X):
        y=[]
        count=0
        for x in X.toarray():
            max_class_probab = float('-inf')
            most_probab_class=''
            for k in self.classes:
                likelihood = 0.0
                #TODO: conver this to a vector summation
                liklihood_v = x*np.log(self.thetajk[k]) + (1-x)*(np.log(1-self.thetajk[k]))
                curr_class_probab = liklihood_v.sum() + np.log(self.thetak[k])
                if curr_class_probab > max_class_probab:
                    max_class_probab = curr_class_probab
                    most_probab_class = k
                    
            y.append(most_probab_class)
        
        return np.array(y)

    def score(self, X, y):
        y_pred = self.predict(X)
        
        score = (y_pred==y).sum()/float(y.shape[0])
        return score
        

In [857]:
cmultiNB = MultiNomialNB()

In [858]:
cv = CountVectorizer(binary=True, stop_words='english')
x_train_v = cv.fit_transform(x_train[:3])
x_test_v = cv.transform(x_test[:3])

In [859]:
cmultiNB.fit(x_train_v, y_train[:3])

defaultdict(<class 'float'>, {'nfl': 0.3333333333333333, 'hockey': 0.3333333333333333, 'GlobalOffensive': 0.3333333333333333}) 25


In [860]:
cmultiNB.score(x_test_v, pd.Series(['GlobalOffensive', 'l', 'l']))

0.3333333333333333

In [876]:
x_test_v[::-1]
y_test.unique()[0]

'baseball'

In [976]:
a= np.matrix([0]*10)

In [978]:
a[1,0]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [1201]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

redditDataTrain = pd.read_csv("data/reddit_train.csv") #, sep="\n", header=None) 
redditDataTest = pd.read_csv("data/reddit_test.csv") # sep="\n", header=None)

redditDataTrain = redditDataTrain

commentsTrain = redditDataTrain.iloc[:,1]
subredditsTrain = redditDataTrain.iloc[:,-1]
commentsTest = redditDataTest.iloc[:,1]

x_train, x_test, y_train, y_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2, random_state=4)

cv = CountVectorizer(binary=True, stop_words='english')
x_train_v = cv.fit_transform(commentsTrain[:4].append(commentsTrain[:4], ignore_index=True))
x_test_v = cv.transform(commentsTrain[:2])

In [1126]:
X = x_train_v
y = subredditsTrain[:4].append(subredditsTrain[99:103], ignore_index=True)
print(y)
thetak=defaultdict(float)
classes = y.unique()
thetajk = np.arange(len(classes)*X.shape[1], dtype=float).reshape(len(classes),X.shape[1])

for i,k in enumerate(classes[:2]):
    thetak[k] = (y==k).sum()/float(y.shape[0])
    indices = np.where(y==k)[0]
    print('i', indices)
    filteredX = X[indices]
    print("f", filteredX.toarray())
    thetajk[i] = (filteredX.sum(axis=0)+0.01)/float(filteredX.shape[0]+0.02)
    print(thetajk[i])
print(thetak, thetajk.shape)

0             hockey
1                nba
2    leagueoflegends
3             soccer
4         conspiracy
5                wow
6              funny
7          AskReddit
Name: subreddits, dtype: object
i [0]
f [[1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 0
  1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0]]
[0.99019608 0.00980392 0.00980392 0.00980392 0.00980392 0.00980392
 0.99019608 0.00980392 0.99019608 0.99019608 0.00980392 0.99019608
 0.99019608 0.00980392 0.00980392 0.99019608 0.99019608 0.00980392
 0.99019608 0.00980392 0.99019608 0.00980392 0.00980392 0.99019608
 0.99019608 0.00980392 0.00980392 0.99019608 0.00980392 0.00980392
 0.99019608 0.00980392 0.00980392 0.99019608 0.99019608 0.00980392
 0.99019608 0.00980392 0.00980392 0.99019608 0.00980392 0.99019608
 0.99019608 0.00980392 0.00980392 0.99019608 0.99019608 0.99019608
 0.99019608 0.99019608 0.00980392 0.00980392 0.99019608 0.99019608
 0.00980392 0.99019608 0.00980392 0.99019608 0.00980392 0.00980

In [1310]:
X = x_test_v
print(X.shape, thetajk.shape)
# right = (1-X.toarray()).dot(np.subtract(1, np.log(thetajk).T))
left = X.dot(np.log(thetajk).T)
middle = np.array([1]*X.shape[0]*X.shape[1]).reshape(X.shape).dot(np.log(1-thetajk).T)
right = X.dot(np.log(1-thetajk).T)
temp = np.add(left, middle, right)
temp.shape
# left.shape, middle.shape, right.shape

(2, 60) (8, 60)


/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  


(2, 8)

In [1242]:
type(np.array([1]*X.shape[0]*X.shape[1]).reshape(X.shape[0],X.shape[1]))

numpy.ndarray